In [ ]:
import os
import sys  
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Define the path to the src directory
src_path = os.path.abspath('../src')

# Check if the src directory exists
if src_path not in sys.path:
    # Append src directory to the Python path
    sys.path.append(src_path)

from data_processing import valid_loader, train_loader, test_loader, labels
from helper import train, evaluate, test, visualise_all_loss

In [ ]:
num_classes = len(labels)  # Example number of classes

torch.manual_seed(43)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define DenseNet model
dense = models.densenet121(pretrained=True)

# Modify the last layer for multi-class classification
dense.classifier = nn.Sequential(
    nn.Linear(in_features=1024, out_features=len(labels)), # New fully connected layer
    nn.Sigmoid() # Add Sigmoid activation
)

# Freeze the weights of the pre-trained layers
for param in dense.parameters():
    param.requiresGrad = False

# Unfreeze the weights of the last layer
for param in dense.classifier.parameters():
    param.requires_grad = True

dense.to(device)

# Print the model architecture
print(dense)


In [ ]:
# Clear CUDA memory
torch.cuda.empty_cache()

# Paths for saving
save_dir = "../models/densenet_15e/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define Params
criterion = nn.BCELoss()
num_epochs = 15
learning_rates = [0.001, 0.0005]

In [ ]:
# Track losses for visualization
train_losses_dict = {}
valid_losses_dict = {}

# Iterate over different learning rates
for lr in learning_rates:
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, dense.parameters()), lr=lr)

    best_valid_loss = float('inf')
    train_losses = []  
    valid_losses = []  
    
    for epoch in range(num_epochs):
        avg_train_loss = train(dense, train_loader, optimizer, criterion, device)
        train_losses.append(avg_train_loss)
        
        valid_loss = evaluate(dense, valid_loader, criterion, device)
        valid_losses.append(valid_loss)
        
        # Print validation loss
        print(f'Learning Rate: {lr}, Epoch: {epoch+1}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {valid_loss:.4f}')
        
        # Save the best model if validation loss improves
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(dense.state_dict(), os.path.join(save_dir, f'best_model_lr_{lr}.pt'))

    # Store losses for visualization
    train_losses_dict[lr] = train_losses
    valid_losses_dict[lr] = valid_losses

# Save losses dictionaries for visualization later
torch.save(train_losses_dict, os.path.join(save_dir, 'train_losses.pt'))
torch.save(valid_losses_dict, os.path.join(save_dir, 'valid_losses.pt'))

In [ ]:
# Evaluate Model
for lr in learning_rates:
    model_path = os.path.join(save_dir, f'best_model_lr_{lr}.pt')  # Change the filename accordingly
    dense.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    metrics, loss = test(dense, test_loader, criterion, device, num_classes)
    print(metrics, loss)

In [ ]:
# Plot Train Loss and Validation Loss on the same graph
train_model_path = os.path.join(save_dir, f'train_losses.pt')
valid_model_path = os.path.join(save_dir, f'valid_losses.pt')

visualise_all_loss(train_model_path, valid_model_path)